In [10]:
%load_ext autoreload
%autoreload 2

from visualiser.experiment_ui import ExperimentUI

ui = ExperimentUI()

ui.get_ui_object()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Continuing overwrite
Loading agent <class 'agents.agents.ddpg.TD3Agent'>
Setting reward <function simple_reward at 0x00000146E43B2790>
Setting environment ./data/citylearn_challenge_2022_phase_1/schema.json
